 # 0. Naive Model
 
Use the realized volatility at of the current 10 min time interval as the predict value of the realized volatility at of the next 10 min. Reproduce the process as the one in the  tutorial notebook.

## 0.1 Predict the volatility of stock_0 at time 5

In [10]:
import numpy as np
import pandas as pd
import plotly.express as px


book_train_0 = pd.read_parquet('data/optiver-realized-volatility-prediction/book_train.parquet/stock_id=0')
trade_train_0 = pd.read_parquet('data/optiver-realized-volatility-prediction/trade_train.parquet/stock_id=0')

train = pd.read_csv('data/optiver-realized-volatility-prediction/train.csv')

stock_id = 0
book_train_example = book_train_0[book_train_0['time_id']==11].copy()
book_train_example['stock_id'] = stock_id

    
book_train_example.head()


,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2,stock_id
302,11,0,0.999473,1.000176,0.999423,1.000376,205,100,20,30,0
303,11,3,0.999473,1.000176,0.999423,1.000376,200,100,20,30,0
304,11,13,0.999473,1.000326,0.999423,1.000376,200,100,20,30,0
305,11,18,0.999473,1.000025,0.999423,1.000376,200,100,20,30,0
306,11,28,0.999473,1.000326,0.999423,1.000376,200,100,20,30,0


In [11]:
book_train_example['wap'] = (book_train_example['bid_price1'] * book_train_example['ask_size1'] +
                                book_train_example['ask_price1'] * book_train_example['bid_size1']) / (
                                       book_train_example['bid_size1']+ book_train_example['ask_size1'])
fig = px.line(book_train_example, x="seconds_in_bucket", y="wap", title='WAP of stock_id_0, time_id_5')
fig.show()

In [12]:
book_train_example['log_return'] = np.log(book_train_example['wap'])-np.log(book_train_example['wap'].shift(1))
book_train_example.dropna(subset=['log_return'])

fig1 = px.line(book_train_example, x="seconds_in_bucket", y="log_return", title='WAP of stock_id_0, time_id_5')
fig1.show()

In [13]:
book_train_example

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2,stock_id,wap,log_return
302,11,0,0.999473,1.000176,0.999423,1.000376,205,100,20,30,0,0.999945,NaN
303,11,3,0.999473,1.000176,0.999423,1.000376,200,100,20,30,0,0.999941,-0.000004
304,11,13,0.999473,1.000326,0.999423,1.000376,200,100,20,30,0,1.000042,0.000100
305,11,18,0.999473,1.000025,0.999423,1.000376,200,100,20,30,0,0.999841,-0.000201
306,11,28,0.999473,1.000326,0.999423,1.000376,200,100,20,30,0,1.000042,0.000201
...,...,...,...,...,...,...,...,...,...,...,...,...,...
497,11,570,1.000025,1.000778,0.999975,1.000878,309,10,120,44,0,1.000754,0.000000
498,11,574,1.000025,1.000778,0.999975,1.000878,200,10,120,44,0,1.000742,-0.000012
499,11,584,1.000276,1.000778,0.999975,1.000878,1,32,120,84,0,1.000291,-0.000451
500,11,585,1.000276,1.000778,0.999975,1.000878,1,32,120,44,0,1.000291,0.000000


In [14]:
def realized_volatility(log_returns):
    return np.sqrt(np.sum(log_returns**2))

realized_vol = realized_volatility(book_train_example['log_return'])

print(f'The predicted realized volatility value for stock 0 at time 5 is {realized_vol}.')

The predicted realized volatility value for stock 0 at time 5 is 0.0012044312600105803.


In [9]:
realized_vol

0.004499364172786559

## 0.2 Naive model

In [ ]:
import glob
from sklearn.metrics import r2_score

list_order_book_train = glob.glob(
    'optiver-realized-volatility-prediction/book_train.parquet/*')


def get_volatility_prediction(file_path):
    book_train = pd.read_parquet(file_path)
    book_train['wap'] = (book_train['bid_price1'] * book_train['ask_size1'] +
                         book_train['ask_price1'] * book_train['bid_size1']) / (
        book_train['bid_size1'] + book_train['ask_size1'])

    book_train['log_return'] =book_train.groupby(['time_id'])['wap'].transform(lambda x: np.log(x) - np.log(x.shift(1)))
    
    book_train.dropna(subset=['log_return'])
    
    prediction_pd= pd.DataFrame(book_train.groupby(['time_id'])[
                              'log_return'].agg(realized_volatility)).reset_index()
    prediction_pd = prediction_pd.rename(columns={'log_return':'pred'})
    stock_id = file_path.split('=')[1]
    prediction_pd['row_id']=prediction_pd['time_id'].apply(lambda x: f'{stock_id}-{x}')
    
    return prediction_pd[['row_id','pred']]

def all_realized_volatility(file_list):
    df_pred = pd.DataFrame()
    for file_path in file_list:
        df_pred = pd.concat([df_pred,get_volatility_prediction(file_path)],ignore_index=True)
        

df_realized_pred_train = all_realized_volatility(list_order_book_train)

    

In [46]:
def rmspe(y_true, y_pred):
    return  (np.sqrt(np.mean(np.square((y_true - y_pred) / y_true))))

train.head()

,stock_id,time_id,target
0,0,5,0.004136
1,0,11,0.001445
2,0,16,0.002168
3,0,31,0.002195
4,0,62,0.001747


In [47]:
train['row_id'] = train['stock_id'].astype(str) + '-' + train['time_id'].astype(str)
train = train[['row_id','target']]
df_joined = train.merge(df_realized_pred_train[['row_id','pred']], on = ['row_id'], how = 'left')

In [48]:
R2 = round(r2_score(y_true = df_joined['target'], y_pred = df_joined['pred']),3)
RMSPE = round(rmspe(y_true = df_joined['target'], y_pred = df_joined['pred']),3)
print(f'Performance of the naive prediction: R2 score: {R2}, RMSPE: {RMSPE}')

Performance of the naive prediction: R2 score: 0.628, RMSPE: 0.341
